In [ ]:
import pandas as pd
import numpy as np
import geopandas as gp

In [ ]:
households_csv = '~/Data/UKDA-5340-tab/tab/householdeul2017.tab'
individuals_csv = '~/Data/UKDA-5340-tab/tab/individualeul2017.tab'
trips_csv ='~/Data/UKDA-5340-tab/tab/tripeul2017.tab'
"https://geoportal.statistics.gov.uk/datasets/counties-and-unitary-authorities-december-2017-full-extent-boundaries-in-uk-wgs84?geometry=-1.922%2C50.793%2C2.379%2C51.397&page=2"
areas_shp = '/Users/fred.shone/Data/UKDA-5340-tab/Counties_and_Unitary_Authorities__December_2017__Boundaries_in_the_UK__WGS84/'

## Load households data

In [ ]:
hh_in = pd.read_csv(
    households_csv,
    sep='\t',
    usecols=['HouseholdID', 'SurveyYear', 'PSUID', 'W2', 'OutCom_B02ID',
       'HHIncome2002_B02ID', 'AddressType_B01ID', 'Ten1_B02ID',
       'Landlord_B01ID', 'ResLength_B01ID', 'HHoldCountry_B01ID',
       'HHoldGOR_B02ID', 'HHoldNumAdults', 'HHoldNumChildren',
       'HHoldNumPeople', 'HHoldStruct_B02ID', 'NumLicHolders',
       'HHoldEmploy_B01ID', 'NumVehicles', 'NumBike', 'NumCar', 'NumMCycle',
       'NumVanLorry', 'NumCarVan', 'WalkBus_B01ID', 'Getbus_B01ID',
       'WalkRail_B01ID', 'WalkRailAlt_B01ID',
       'HRPWorkStat_B02ID', 'HRPSEGWorkStat_B01ID', 'HHoldOAClass2011_B03ID',
       'Settlement2011EW_B03ID', 'Settlement2011EW_B04ID'],
)

hh_in.HHIncome2002_B02ID = pd.to_numeric(hh_in.HHIncome2002_B02ID, errors='coerce')
hh_in.NumLicHolders = pd.to_numeric(hh_in.NumLicHolders, errors='coerce')
hh_in.NumVehicles = pd.to_numeric(hh_in.NumVehicles, errors='coerce')
hh_in.NumCar = pd.to_numeric(hh_in.NumCar, errors='coerce')
hh_in.NumMCycle = pd.to_numeric(hh_in.NumMCycle, errors='coerce')
hh_in.NumVanLorry = pd.to_numeric(hh_in.NumVanLorry, errors='coerce')
hh_in.NumCarVan = pd.to_numeric(hh_in.NumCarVan, errors='coerce')
hh_in.Settlement2011EW_B04ID = pd.to_numeric(hh_in.Settlement2011EW_B04ID, errors='coerce')

hh_in.head()

In [ ]:
participation_mapping = dict(zip(hh_in.HouseholdID, hh_in.OutCom_B02ID))
weight_mapping = dict(zip(hh_in.HouseholdID, hh_in.W2))

## Load person data

In [ ]:
persons_in = pd.read_csv(
    individuals_csv,
    sep='\t',
    usecols=['SurveyYear', 'IndividualID', 'HouseholdID', 'PSUID', 'VehicleID',
       'PersNo', 'Age_B01ID', 'OfPenAge_B01ID', 'Sex_B01ID', 'EdAttn1_B01ID',
       'EdAttn2_B01ID', 'EdAttn3_B01ID', 'DrivLic_B02ID', 'CarAccess_B01ID',
       'DrivDisable_B01ID', 'WkPlace_B01ID', 'ES2000_B01ID', 'NSSec_B03ID',
       'SC_B01ID', 'Stat_B01ID', 'SVise_B01ID', 'EcoStat_B02ID',
       'PossHom_B01ID']
)
persons_in.head()

## load trip data
Pos. = 1	Variable = TripID	Variable label = Trip unique ID - Created in SQL
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for TripID

Pos. = 2	Variable = SurveyYear	Variable label = Survey year - actual year
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for SurveyYear

Pos. = 3	Variable = DayID	Variable label = ID given to all trips made by an individual on a given travel day - Created in SQL
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for DayID

Pos. = 4	Variable = IndividualID	Variable label = Individual unique ID - Created in SQL
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for IndividualID

Pos. = 5	Variable = HouseholdID	Variable label = Household unique ID - Created in SQL
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for HouseholdID

Pos. = 6	Variable = PSUID	Variable label = PSU unique ID - Created in SQL
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for PSUID

Pos. = 7	Variable = PersNo	Variable label = Person number within the household
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for PersNo

Pos. = 8	Variable = TravDay	Variable label = Day of the travel week (1-7)
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for TravDay

Pos. = 9	Variable = JourSeq	Variable label = Journey number on a given travel day
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for JourSeq


In [ ]:
travel_diaries_in = pd.read_csv(
    trips_csv,
    sep='\t',
    usecols=['TripID', 'SurveyYear', 'DayID', 'IndividualID', 'HouseholdID', 'PSUID',
       'PersNo', 'TravDay', 'JourSeq', 'ShortWalkTrip_B01ID', 'NumStages',
       'MainMode_B04ID', 'TripPurpFrom_B01ID',
        'TripPurpTo_B01ID', 'TripPurpose_B04ID',
       'TripStart', 'TripEnd', 'TripOrigUA2009_B01ID', 'TripDestUA2009_B01ID'],
#     dtype={"W5": np.float64,}
)

travel_diaries_in.TripStart = pd.to_numeric(travel_diaries_in.TripStart, errors='coerce')
travel_diaries_in.TripEnd = pd.to_numeric(travel_diaries_in.TripEnd, errors='coerce')

travel_diaries_in.head()

In [ ]:
travel_diaries_in['participation'] = travel_diaries_in.HouseholdID.map(participation_mapping)
travel_diaries_in['hh_weight'] = travel_diaries_in.HouseholdID.map(weight_mapping)

In [ ]:
travel_diaries = travel_diaries_in.loc[travel_diaries_in.participation.isin([1,2])]

## PAM schema
    pid - person ID
    hid - household ID
    seq - trip sequence number
    hzone - household zone
    ozone - trip origin zone
    dzone - trip destination zone
    purp - trip purpose
    mode - trip mode
    tst - trip start time (minutes)
    tet - trip end time (minutes)
    freq - weighting for representative population

In [ ]:
travel_diaries.rename(
    columns={  # rename data
        'JourSeq': 'seq',
        'TripOrigUA2009_B01ID': 'ozone',
        'TripDestUA2009_B01ID': 'dzone',
        'TripPurpFrom_B01ID': 'oact',
        'TripPurpTo_B01ID': 'dact',
        'MainMode_B04ID': 'mode',
        'TripStart': 'tst',
        'TripEnd': 'tet',
    },
                inplace=True)

travel_diaries.head()

In [ ]:
travel_diaries.dtypes

In [ ]:
def check_uniques(df):
    for c in df.columns:
        print(c)
        n = df[c].nunique()
        if n < 1000:
            print(df[c].unique())

In [ ]:
check_uniques(travel_diaries)

## Clean out incomplete plans

In [ ]:
def remove_broken_plans(plan):
    if plan.isnull().values.any():
        return None
    for col in ['ozone', 'dzone']:
        if -8 in list(plan[col]):
            return None
    return plan

In [ ]:
clean_travel_diaries = travel_diaries.groupby(
    ['IndividualID', 'TravDay']
).apply(
    remove_broken_plans
).reset_index(drop=True)

In [ ]:
clean_travel_diaries.head()

In [ ]:
print(len(travel_diaries))
print(len(clean_travel_diaries))

## Reweight and Split Days

In [ ]:
# reweight and split ids for unique days

def reweight(group):
    """
    Reweight based on multiple diary days, ie if an agent has two diary days, we will treat these as
    two unique agents, so we half the original weighting
    """
    group['freq'] = group.hh_weight / group.DayID.nunique()
    return group

trips = clean_travel_diaries.groupby('IndividualID').apply(reweight)
trips['pid'] = [f"{p}_{d}" for p, d in zip(trips.IndividualID, trips.TravDay)]
trips['hid'] = [f"{h}_{d}" for h, d in zip(trips.HouseholdID, trips.TravDay)]

In [ ]:
trips.head()

In [ ]:
def expand_days(
    trips,
    target,
    trips_on='Diary_number',
    target_on='Diary_number',
    new_id='pid',
    trim=True
):
    """
    Expand target df based on mapping between trips traget_on and new_id.
    Set index to new_id.
    """
    print("Building mapping.")
    mapping = {}
    for i, person in trips.groupby(target_on):
        mapping[i] = list(set(person[new_id]))
    n = len(mapping)
    
    if trim:
        print("Trimming target.")
        selection = set(trips[trips_on])
        target = target.loc[target[target_on].isin(selection)]
    
    expanded = pd.DataFrame()
    for p, (i, ids) in enumerate(mapping.items()):
        if not p % 10:
            print(f"Building expanded data {p}/{n}", end='\r', flush=True)
        for idx in ids:
            split = target.loc[target[target_on] == i]
            split[new_id] = idx
            expanded = expanded.append(split)
    expanded.set_index(new_id, inplace=True)
    print(f"Done")
    return expanded

In [ ]:
hhs = expand_days(
    trips,
    hh_in,
    trips_on='HouseholdID',
    target_on='HouseholdID',
    new_id='hid'
)

In [ ]:
hhs.head()

In [ ]:
people = expand_days(
    trips,
    persons_in,
    trips_on='IndividualID',
    target_on='IndividualID',
    new_id='pid'
)

In [ ]:
people.head()

## Build Mappings and apply to common fields

In [ ]:
def string_to_dict(string):
    """used to build dicts from cut and paste rtf dictionaries"""
    mapping = {}
    for line in string.split("\n"):
        _, v, l = line.split("\t")
        v = v.split(" = ")[1]
        l = l.split(" = ")[1]
        mapping[float(v)] = str(l)
    return mapping

In [ ]:
mode_mapping = {
    1: 'Walk',
     2: 'Bicycle',
     3: 'Car/van driver',
     4: 'Car/van passenger',
     5: 'Motorcycle',
     6: 'Other private transport',
     7: 'Bus', #Bus in London',
     8: 'Bus', #'Other local bus',
     9: 'Coach', #'Non-local bus',
     10: 'London Underground',
     11: 'Surface Rail',
     12: 'Taxi/minicab',
     13: 'Other public transport',
     -10: 'DEAD',
     -8: 'NA'
}

In [ ]:
purp_mapping = {
    1: 'Work',
     2: 'In course of work',
     3: 'Education',
     4: 'Food shopping',
     5: 'Non food shopping',
     6: 'Personal business medical',
     7: 'Personal business eat/drink',
     8: 'Personal business other',
     9: 'Eat/drink with friends',
     10: 'Visit friends',
     11: 'Other social',
     12: 'Entertain/ public activity',
     13: 'Sport: participate',
     14: 'Holiday: base',
     15: 'Day trip/just walk',
     16: 'Other non-escort',
     17: 'Escort home',
     18: 'Escort work',
     19: 'Escort in course of work',
     20: 'Escort education',
     21: 'Escort shopping/personal business',
     22: 'Other escort',
     23: 'Home',
     -10: 'DEAD',
     -8: 'NA'
}

In [ ]:
trips['mode'] = trips['mode'].map(mode_mapping)
trips['oact'] = trips['oact'].map(purp_mapping)
trips['dact'] = trips['dact'].map(purp_mapping)

## Area Mapping - NOT WORKING

In [ ]:
area_mapping = {
    520.0: 'Warwickshire',
     530.0: 'West Midlands',
     540.0: 'West Sussex',
     191.0: 'Derby',
     550.0: 'West Yorkshire',
     560.0: 'Wiltshire',
     561.0: 'Swindon',
     601.0: 'Isle of Anglesey',
     602.0: 'Gwynedd',
     603.0: 'Conwy',
     604.0: 'Denbighshire',
     605.0: 'Flintshire',
     606.0: 'Wrexham',
     607.0: 'Powys',
     608.0: 'Ceredigion',
     609.0: 'Pembrokeshire',
     610.0: 'Carmarthenshire',
     611.0: 'Swansea',
     612.0: 'Neath and Port Talbot',
     101.0: 'Bath and N.E. Somerset',
     614.0: 'Vale of Glamorgan',
     103.0: 'North Somerset',
     616.0: 'Rhondda, Cynon, Taff',
     617.0: 'Merthyr Tydfil',
     618.0: 'Caerphilly',
     615.0: 'Cardiff',
     620.0: 'Torfaen',
     621.0: 'Monmouthshire',
     622.0: 'Newport',
     111.0: 'Luton',
     112.0: 'Bedford',
     104.0: 'South Gloucestershire',
     190.0: 'Derbyshire',
     121.0: 'Bracknell Forest',
     122.0: 'Newbury',
     123.0: 'Reading',
     124.0: 'Slough',
     125.0: 'Windsor & Maidenhead',
     126.0: 'Wokingham',
     130.0: 'Buckinghamshire',
     131.0: 'Milton Keynes',
     140.0: 'Cambridgeshire',
     141.0: 'Peterborough',
     151.0: 'Halton',
     152.0: 'Warrington',
     153.0: 'Cheshire East',
     154.0: 'Cheshire West and Chester',
     161.0: 'Hartlepool',
     162.0: 'Middlesbrough',
     163.0: 'Redcar & Cleveland',
     164.0: 'Stockton-on-Tees',
     113.0: 'Bedfordshire, Central',
     170.0: 'Cornwall & Isles of Scilly',
     200.0: 'Devon',
     180.0: 'Cumbria',
     201.0: 'Plymouth',
     701.0: 'Aberdeen City',
     702.0: 'Aberdeenshire',
     703.0: 'Angus',
     704.0: 'Argyll and Bute',
     705.0: 'Scottish Borders',
     706.0: 'Clackmannanshire',
     707.0: 'West Dunbartonshire',
     708.0: 'Dumfries and Galloway',
     709.0: 'Dundee, City of',
     710.0: 'East Ayrshire',
     711.0: 'East Dunbartonshire',
     712.0: 'East Lothian',
     713.0: 'East Renfrewshire',
     714.0: 'Edinburgh, City of',
     715.0: 'Falkirk',
     716.0: 'Fife',
     717.0: 'Glasgow, City of',
     718.0: 'Highland',
     719.0: 'Inverclyde',
     720.0: 'Midlothian',
     721.0: 'Moray',
     210.0: 'Dorset',
     211.0: 'Bournemouth',
     212.0: 'Poole',
     725.0: 'Perth and Kinross',
     726.0: 'Renfrewshire',
     727.0: 'Shetland Islands',
     728.0: 'South Ayrshire',
     729.0: 'South Lanarkshire',
     730.0: 'Stirling',
     731.0: 'West Lothian',
     732.0: 'Western Isles',
     221.0: 'Darlington',
     230.0: 'East Sussex',
     231.0: 'Brighton and Hove',
     722.0: 'North Ayrshire',
     240.0: 'Essex - area outside M25',
     241.0: 'Southend on Sea',
     242.0: 'Thurrock',
     723.0: 'North Lanarkshire',
     724.0: 'Orkney Islands',
     250.0: 'Gloucestershire',
     202.0: 'Torbay',
     260.0: 'Greater Manchester',
     270.0: 'Hampshire',
     271.0: 'Portsmouth',
     272.0: 'Southampton',
     280.0: 'Worcestershire',
     281.0: 'Herefordshire',
     800.0: 'Inner London - excluding Central London',
     290.0: 'Hertfordshire - area outside M25',
     220.0: 'Durham',
     810.0: 'Essex - area within M25',
     301.0: 'East Riding of Yorkshire',
     302.0: 'Kingston upon Hull, City of',
     303.0: 'North East Lincolnshire',
     304.0: 'North Lincolnshire',
     820.0: 'Hertfordshire - area within M25',
     310.0: 'Isle of Wight',
     830.0: 'Kent - area within M25',
     320.0: 'Kent - area outside M25',
     321.0: 'Medway Towns',
     840.0: 'Surrey - area within M25',
     330.0: 'Lancashire',
     331.0: 'Blackburn with Darwen',
     332.0: 'Blackpool',
     340.0: 'Leicestershire',
     341.0: 'Leicester',
     342.0: 'Rutland',
     350.0: 'Lincolnshire',
     360.0: 'London Central',
     370.0: 'Outer London',
     380.0: 'Merseyside',
     390.0: 'Norfolk',
     400.0: 'Northamptonshire',
     410.0: 'Northumberland',
     420.0: 'North Yorkshire',
     421.0: 'York',
     430.0: 'Nottinghamshire',
     431.0: 'Nottingham',
     440.0: 'Oxfordshire',
     450.0: 'Shropshire',
     451.0: 'The Wrekin',
     460.0: 'Somerset',
     470.0: 'South Yorkshire',
     480.0: 'Staffordshire',
     481.0: 'Stoke-on-Trent',
     490.0: 'Suffolk',
     613.0: 'Bridgend',
     619.0: 'Blaenau Gwent',
     500.0: 'Surrey - area outside M25',
     -10.0: 'DEAD',
     -9.0: 'DNA',
     -8.0: 'NA',
     102.0: 'Bristol, City of',
     510.0: 'Tyne and Wear'
}

In [ ]:
# areas = gp.read_file(areas_shp)
# areas.head()

In [ ]:
# area_code_mapping = dict(zip(areas.ctyua17nm, areas.ctyua17cd))

In [ ]:
# code_mapping = {k: area_code_mapping[v] for k, v in area_mapping.items()}

In [ ]:
# trips.rename(columns={  # rename data
#     'opurp': 'oact',
#     'dpurp': 'dact',
# },
#             inplace=True)

In [ ]:
trips.head()

In [ ]:
trips.loc[trips.pid == '2016000001_1']

## Load into PAM

In [ ]:
from pam import write
from pam import read

In [ ]:
trips.tst = trips.tst.astype(int)
trips.tet = trips.tet.astype(int)

In [ ]:
population = read.load_travel_diary_from_to(
    trip_diary=trips,
    person_attributes=people,
    hh_attributes=hhs,
    sample_perc=.01
)

In [ ]:
population.fix_plans()

In [ ]:
population.size

In [ ]:
population.stats

In [ ]:
from pam.plot.stats import plot_activity_times, plot_leg_times

In [ ]:
plot_activity_times(population)

In [ ]:
plot_leg_times(population)

In [ ]:
population.to_csv("~/Data/prelimDfT")

In [ ]:
population.random_household().print()

In [ ]:
population.random_household().plot()